In [24]:

from importlib import reload
import plot, utils, data_load, clustering, constants 
import warnings
import random

# use reload whenever make changes to underlying file
# modules_to_reload = [plot, utils, data_load, clustering, constants ]

# for module in modules_to_reload:
#     reload(module)


In [11]:
# Suppress FutureWarnings from multiple modules
modules_to_ignore = ("seaborn", "sklearn")

for module in modules_to_ignore:
    warnings.filterwarnings("ignore", category=FutureWarning, module=module)

In [3]:
maf_df = data_load.load_maf_data(file_path=constants.maf_file_path)
expression_df = data_load.load_txt_file_into_dataframe(file_path=constants.expression_file_path)
expression_df_melted = data_load.reformat_expression_data(df=expression_df)
mutation_expression_df_melted = data_load.preprocess_and_combine_mutation_expression(maf_df= maf_df, expression_df = expression_df_melted)

fraction of rows filtered is 0.2865853658536585


In [43]:
reload(clustering)
reload(utils)
reload(data_load)
scores_genes = {}
for target_gene in constants.genes:  
    # target_gene = random.choice(constants.genes). use when testing

    # calculate logfc, pvalue
    volcano_plot_df, individuals_mutated_target_gene, volcano_input_filename = utils.generate_stats_per_gene(
        express_mut_genes_df=mutation_expression_df_melted, 
        target_gene=target_gene,
        output_folder=target_gene)

    # get top 100 genes with the lowest p-values
    heatmap_data = data_load.generate_expression_heatmap(expression_df=expression_df, volcano_plot_df=volcano_plot_df, n=100)

    # cluster top 100 genes and samples
    # evaluate clustering with sihlouttee 
    # filter to only get silhouttee score for mutatted samples
    row_linkage, col_linkage, row_clusters, col_clusters, row_score, col_score, mutated_col_score = clustering.hierarchical_clustering(expression_df_heatmap=heatmap_data, 
                                                                                                output_folder=target_gene,
                                                                                                row_threshold=7, 
                                                                                                col_threshold=7,
                                                                                                mutated_samples=individuals_mutated_target_gene)
    
    print(f"for target gene {target_gene}  -- row score: {row_score}, col_score: {col_score}, mutated_col_score: {mutated_col_score}")
    

    scores_genes[target_gene]={'gene_sil_score': row_score, 'sample_sil_score': col_score, 'mutated_sample_sil_score': mutated_col_score}



outputting data to FOXA1/15_104_logfc_pvalue.csv
for target gene FOXA1  -- row score: 0.25602165548062367, col_score: 0.04708802972647411, mutated_col_score: 0.06865564802385617
outputting data to GYPB/1_116_logfc_pvalue.csv
for target gene GYPB  -- row score: 0.23421231675385734, col_score: 0.06410185672152817, mutated_col_score: 0.0
outputting data to ID1/1_116_logfc_pvalue.csv
for target gene ID1  -- row score: 0.23923581352611958, col_score: 0.03606137333522512, mutated_col_score: 0.0
outputting data to RPL10/5_112_logfc_pvalue.csv
for target gene RPL10  -- row score: 0.32735862367584057, col_score: 0.03903956075177235, mutated_col_score: 0.10191979322503446
outputting data to SPOP/15_102_logfc_pvalue.csv
for target gene SPOP  -- row score: 0.24124544217953459, col_score: 0.06585128389732592, mutated_col_score: 0.1963834401696408
outputting data to TSPAN9/4_113_logfc_pvalue.csv
for target gene TSPAN9  -- row score: 0.15608305271917275, col_score: 0.022227998832706056, mutated_col_s

In [54]:
reload(utils)
top_genes = utils.output_genes_with_highest_score(scores_genes, n = 10)

# Print the top genes
print("Genes with the Highest Mutated Sample Silhouette Score:")
for i, (gene, scores) in enumerate(top_genes):
    print(f"{i+1}. Gene: {gene}")
    print(f"   Mutated Sample Silhouette Score: {scores['mutated_sample_sil_score']}")
    print(f"   Gene Silhouette Score: {scores['gene_sil_score']}")
    print(f"   Sample Silhouette Score: {scores['sample_sil_score']}")
    print("\n")




Genes with the Highest Mutated Sample Silhouette Score:
1. Gene: ERF
   Mutated Sample Silhouette Score: 0.39251641278388594
   Gene Silhouette Score: 0.20313066882688563
   Sample Silhouette Score: 0.05566518718633649


2. Gene: TRPM1
   Mutated Sample Silhouette Score: 0.3348453681669478
   Gene Silhouette Score: 0.1645550843863594
   Sample Silhouette Score: 0.02277090086582008


3. Gene: NDST2
   Mutated Sample Silhouette Score: 0.3220168796501848
   Gene Silhouette Score: 0.30216158086509837
   Sample Silhouette Score: 0.1017360369448713


4. Gene: TSPAN9
   Mutated Sample Silhouette Score: 0.31293881694406067
   Gene Silhouette Score: 0.15608305271917275
   Sample Silhouette Score: 0.022227998832706056


5. Gene: MED12
   Mutated Sample Silhouette Score: 0.22894002303535443
   Gene Silhouette Score: 0.181637859072286
   Sample Silhouette Score: 0.06157907971023549


6. Gene: SPOP
   Mutated Sample Silhouette Score: 0.1963834401696408
   Gene Silhouette Score: 0.24124544217953459


In [ ]:
# def extract_gene_scores(file_paths, gene_column, score_column):
#     # Initialize an empty dictionary to store gene scores
#     gene_scores = {}

#     for file_path in file_paths:
#         # Read the input file and extract gene names and logFC values
#         with open(file_path, 'r') as file:
#             for line in file:
#                 # Assume the file format is tab-separated
#                 columns = line.strip().split('\t')

#                 # Extract gene and score using specified column indices
#                 gene = columns[gene_column]
#                 score = float(columns[score_column])

#                 # If the gene is already in the dictionary, average the scores
#                 if gene in gene_scores:
#                     gene_scores[gene] = (gene_scores[gene] + score) / 2.0
#                 else:
#                     gene_scores[gene] = score

#     return gene_scores

# # Example usage
# upregulated_file_path = 'Respond/NDST2/signif_genes_positive.csv'
# downregulated_file_path = 'Respond/NDST2/signif_genes_negative.csv'

In [58]:
# Further analysis for top genes
reload(utils)
reload(plot)
reload(clustering)
for i, (target_gene, scores) in enumerate(top_genes):
    print(f'working on target gene {target_gene}')
    mutated_status_df = utils.get_mutated_status(expression_df_heatmap=heatmap_data, 
                                                individuals_mutated_target_gene=individuals_mutated_target_gene, 
                                                output_folder=target_gene)

    # plot clustered heatmap
    clustered_heatmap = plot.create_clustered_heatmap_and_save(heatmap_data, 
                                                          row_linkage, 
                                                          col_linkage,
                                                          mutated_status_df,
                                                          output_folder=target_gene)

    # plot truncated dendograms 
    row_dendrogram, col_dendrogram = clustering.plot_and_save_dendrograms(row_linkage, 
                                                               col_linkage, 
                                                               heatmap_data, 
                                                               output_folder=target_gene)

    # plot histograms of adjusted and regular pvalues 
    plot.histogram_of_column_and_save(volcano_plot_df, 'pvalue', output_folder=target_gene)
    plot.histogram_of_column_and_save(volcano_plot_df, 'adjusted_pvalue', output_folder=target_gene)

    # calculate significant genes determined by cutoffs
    significant_genes_positive, significant_genes_negative  = plot.volcano_plot(input_file_path=volcano_input_filename, 
                                                                            yaxis='pvalue', 
                                                                            xaxis='logFC', 
                                                                            output_folder=target_gene,
                                                                            significance_threshold=0.05, 
                                                                            logfold_positive_threshold=2, 
                                                                            logfold_negative_threshold=-2)

    # create box plots of significant genes (uoregulated and downregulated genes)
    plot.create_gene_expression_boxplot(expression_df, significant_genes_positive, mutated_status_df, positive=1, output_folder=target_gene)
    plot.create_gene_expression_boxplot(expression_df, significant_genes_negative, mutated_status_df, positive=0, output_folder=target_gene)

    break
    

,gene,score
0,FAM163A,-2.044401
1,HSFX3,-23.439849
2,KLF17,-2.541747
3,KRTAP4-3,-4.495554
4,PIGY,-2.758774
5,PRAME,-3.016260
6,RSC1A1,-16.599572
7,SERPINA11,-2.512787
8,FCRL4,3.185335
9,MUC22,2.751783


In [ ]:
# Reset warnings to default behavior after the code block
warnings.resetwarnings()